In [1]:
import openpathsampling as paths

# New Network Input in OpenPathSampling

This notebook includes some demonstrations of the features that were added by the E-CAM module that created an improved approach to network creation in OpenPathSampling. The improved approach has two main subunits:

* Improved treatment of interface sets
* Improved treatment of multiple state outer interface

To show some examples, I will create a simple collective variable to use as the basis of tests.

In [2]:
cv = paths.CoordinateFunctionCV("x", lambda snap: snap.xyz[0][0])

## Interface sets

There are a few points to consider about the definition of interface sets. The most common case is that each interface volume is defined by the same CV, with different maximum values of the CV, but the same minimum value. However, it is also possible to have interface sets where both minimum and maximum value are changing (the value of $\lambda$ is not uniquely defined by inspecting the `Volume` objects) or where the interfaces don't all grow in terms of a single common CV.

The latter two examples still aren't really supported by our analysis, but we want to make sure that it is possible to support them in the future. However, it would be helpful to make the most common use cases simple.

### Ways of creating the new InterfaceSet

The most common way to create the interface set is barely changed from the previous version. The only difference that is that we use the `VolumeInterfaceSet` object, instead of the `VolumeFactory.CVRangeVolumeSet` function.

In [3]:
interface_set = paths.VolumeInterfaceSet(cv=cv, minvals=0.0, maxvals=[0.2, 0.3, 0.4])

There are other ways to create the interface set. One approach, which is essentially equivalent, would be:

In [4]:
vol1 = paths.CVDefinedVolume(cv, 0.0, 0.2)
vol2 = paths.CVDefinedVolume(cv, 0.0, 0.3)
vol3 = paths.CVDefinedVolume(cv, 0.0, 0.4)

interface_set_2 = paths.InterfaceSet(volumes=[vol1, vol2, vol3],
                                     cv=cv,
                                     lambdas=[0.2, 0.3, 0.4])

We can also create other interface sets, which don't have unique values of $\lambda$ associated:

In [5]:
interface_set_3 = paths.VolumeInterfaceSet(cv=cv, minvals=[-0.1, -0.2, -0.3], maxvals=[0.1, 0.2, 0.3])

In [6]:
# equivalent to interface_set_3
vol1 = paths.CVDefinedVolume(cv, -0.1, 0.1)
vol2 = paths.CVDefinedVolume(cv, -0.2, 0.2)
vol3 = paths.CVDefinedVolume(cv, -0.3, 0.3)
interface_set_4 = paths.InterfaceSet(volumes=[vol1, vol2, vol3], cv=cv)

We can also force all of the volumes created by a `VolumeInterfaceSet` to be intersected with some fixed volume. Of course, doing the same with an `InterfaceSet` (which takes the interface volumes as input, instead of generating them) is trivial.

In [7]:
cv_y = paths.CoordinateFunctionCV("y", lambda snap: snap.xyz[0][1])
volume_y = paths.CVDefinedVolume(cv_y, -1.0, 1.0)

In [8]:
interface_set_5 = paths.VolumeInterfaceSet(cv=cv, minvals=0.0, maxvals=[0.2, 0.3, 0.4], intersect_with=volume_y)

### Associating values of $\lambda$ with the interfaces

One of the major advances here is that we can access the values of the order parameter associated with each interface. This wasn't possible before, and makes analysis much easier.

In [9]:
print interface_set.lambdas

[0.2, 0.3, 0.4]


In [10]:
print interface_set_2.lambdas

[0.2, 0.3, 0.4]


In [11]:
print interface_set_5.lambdas

[0.2, 0.3, 0.4]


Note that not all interface sets have lambda values associated with them. As mentioned above, in some cases, this is literally impossible.

In [12]:
print interface_set_3.lambdas
print interface_set_4.lambdas

None
None


### Creating a new interface for the set

Another new feature is the ability to create a new interface for a given set. This only works for `VolumeInterfaceSet`s.

In [13]:
interface_set.new_interface(lambda_i=0.5)

In [14]:
interface_set_5.new_interface(lambda_i=0.5)

In [15]:
# We print the message of the AttributeError, but ignore the crash
try:
    interface_set_2.new_interface(lambda_i=0.5)
except AttributeError as e:
    print e

'InterfaceSet' object has no attribute 'new_interface'


### Improving analysis due to the new interface sets

Thanks to these new interface sets, we have the ability to identify the volume of lambda for any volume:

In [16]:
volume = interface_set.volumes[0]
interface_set.get_lambda(volume)

0.2

This makes it possible to perform analyses that were previously guesswork. For example, it used to require a complicated algorithm with several assumptions in order to obtain the value of $\lambda$ for the outermost ensemble. Now it can be obtained with:

In [18]:
outermost_lambda = interface_set.get_lambda(interface_set[-1])
print outermost_lambda

0.4


## Multiple State Outer Interface

The other main improvement in this module is the development of a better approach to handle the multiple state outer interface.

### New object to represent MS-outer interface

### Making the MS-outer interface optional